# **Examples for CDC '25**

## **Running Example**

The running example from the paper, illustrating basic concepts in `pyspect`. The task is to verify an overtaking scenario.

### **Imports**

In [6]:
from pyspect import *
from math import pi
from functools import reduce

### **Hyperparameters**

In [7]:
AXIS_NAMES = ['x', 'y', 'yaw', 'vel'] # [m, m, rad, m/s]
MAX_BOUNDS = [200, +4, +pi/16,    25] # pi/12 = 15deg | 35 mps ~= 125 kmph
MIN_BOUNDS = [  0, -4, -pi/16,    15] # pi/12 = 15deg | 15 mps ~= 54 kmph
GRID_SHAPE = (201, 11,     13,    31)

MAX_STEER = pi/128  # [rad/s]   +/- 1.5 degrees/s
MAX_ACCEL = 2.0     # [mps2]    
TIME_STEP = 0.5     # [s]
TIME_HORIZON = 15   # [s]

### **Program**

#### Task definition

1. Define the different regions. These are "constants" in the task specification. Note that we can define the sets lazily, such that we do not rely on a specific implementation or set representation yet. This is possible because `pyspect` internally defers evaluation until realization time.

2. Write the specification $\varphi = (p_\text{hw} \lor p_\text{r}) \:\mathsf{U}\: \texttt{home}$, where $p_\text{hw}, p_\text{r}, \texttt{home} \in \mathsf{AP}$. While $p_\text{hw} \leftrightarrow z \in \texttt{HIGHWAY}$ and $p_\text{r} \leftrightarrow z \in \texttt{RESIDENTIAL}$, we directly use their corresponding sets in the specification in their stead. The proposition $\texttt{home}$ is left symbolic and will be bound later.

In [8]:
## (1) Define the different regions.

RIGHT_LANE  = BoundedSet(y=(-3.5, -0.5))
LEFT_LANE   = BoundedSet(y=(+0.5, +3.5))
BETWEEN     = BoundedSet(y=(-1.5, +1.5))

STRAIGHT = BoundedSet(yaw=(-pi/18, +pi/18))

INFRONT_OBS = HalfSpaceSet(normal=[ +1, -16],
                           offset=[ 40,   1],
                           axes = ['x', 't'])

BEHIND_OBS = HalfSpaceSet(normal=[ -1, +16],
                          offset=[  0,   1],
                          axes = ['x', 't'])

# Funny way of defining the OBS vehicle for two reasons:
# 1. Inter is binary and we have three arguments -> we use functools.reduce
# 2. We define OBS negated from INFRONT_OBS/BEHIND_OBS to make GOAL easier to write
OBS = reduce(Inter, [Compl(INFRONT_OBS), Compl(BEHIND_OBS), RIGHT_LANE])

GOAL = reduce(Inter, [
    INFRONT_OBS, 
    RIGHT_LANE,
    STRAIGHT,
    BoundedSet(x=(..., 195)),
])

## (2) Write the specification.

ENV = reduce(AND, [
    reduce(OR, [
        RIGHT_LANE,
        LEFT_LANE,
        BETWEEN,
    ]),
    NOT(OBS),
    BoundedSet(vel=(16, 24), yaw=(-pi/16 + pi/32, +pi/16 - pi/32)),
])

TASK = UNTIL(ENV, GOAL)

#### Construct the TLT

3. Select the set of primitive TLTs. This determines the logic fragment in which the specification is interpreted. In this example, we use continuous-time LTL, i.e., LTL without the "next" operator.

4. Construct the temporal logic tree and bind the symbolic proposition $\texttt{home}$ to the concrete set $\texttt{H}$. The call to `.where(home=H)` updates a internal proposition map $\mathsf{M}$ so that $\texttt{home} \leftrightarrow z \in H$.

In [9]:
## (3) Select the set of primitive TLTs.

TLT.select(LTLc)

## (4) Create the TLT and set the proposition 'goal'.

objective = TLT(TASK).where(goal=GOAL)

#### Initialize the implementation object

First now, we introduce a specific implementation. `pyspect` has some off-the-shelf implementations that can be imported. Note, however, `pyspect` does not abstract _away_ the underlying back-end completely, as we see in the following cell.

5. Additional imports for the specific implementation. `TVHJImpl` is a wrapper class around the `hj_reachability` library. `pyspect` includes a forked version with convenience utilities.

6. Initialization of the implementation is fully backend-specific. In this case, we specify the dynamics (double integrator), bounds, time horizon, and discretization. This configuration determines how sets like `H` and operators like `UNTIL` will be computed (e.g., using backward reachable sets on a grid).

7. Realize the TLT. This is the key step where the temporal logic and reachability analysis meet: `objective.realize(impl)` recursively applies the reachability and set operations defined by the selected logic fragment and implemented in the backend. The output `out` is the satisfaction set (i.e., the states from which the specification holds) in the implementation's set reprentation. 

8. Plotting. We visualize the satisfaction set using `plot3D_levelset`, which displays the reachable states as a level set over position, velocity, and time. This step is backend-dependent (as value functions are used).

In [10]:
## (5) Additional imports for the specific implementation.

from pyspect.impls.hj_reachability import HJImpl, TVHJImpl
from pyspect.plotting.levelsets import *

import hj_reachability as hj
from hj_reachability.systems import *

## (6) Define the implementation of the reachability algorithm.

dynamics = dict(cls=Bicycle4D,
                wheelbase=2.7,
                min_accel=-MAX_ACCEL,
                max_accel=+MAX_ACCEL,
                min_steer=-MAX_STEER,
                max_steer=+MAX_STEER)

impl = TVHJImpl(dynamics,
                AXIS_NAMES,
                MIN_BOUNDS,
                MAX_BOUNDS,
                GRID_SHAPE,
                TIME_HORIZON,
                time_step=TIME_STEP)

## (7) Run the reachability program ##

# out = objective.realize(impl)
out = TLT(TASK).realize(impl)

# `out` will have the same object type that `impl` operates with.
# For TVHJImpl, `out` will be a numpy array of the gridded value function.
print(f"{type(out) = }, {out.min() = }, {out.max() = }")


## (8) Plotting


ModuleNotFoundError: No module named 'pyspect.impls.plotting'

In [ ]:
plot_levelsets(
    impl.project_onto(out, 't', 'x', 'y', complete=True),
    (impl.project_onto(OBS(impl), 't', 'x', 'y', complete=True), dict(colorscale='reds')),
    plot_func=plot3D_levelset,
    min_bounds=[           0, *MIN_BOUNDS[:2]],
    max_bounds=[TIME_HORIZON, *MAX_BOUNDS[:2]],
    axes=(1, 2, 0),
    xtitle='x (m)',
    ytitle='y (m)',
    ztitle='t (s)',
    eye=EYE_ML_SW,
)

In [ ]:
plot3D_levelset(
    impl.project_onto(out, 't', 'x', 'vel', complete=True),
    min_bounds=[           0, MIN_BOUNDS[0], MIN_BOUNDS[3]],
    max_bounds=[TIME_HORIZON, MAX_BOUNDS[0], MAX_BOUNDS[3]],
    axes=(1, 2, 0),
    xtitle='x (m)',
    ytitle='v (m/s)',
    ztitle='t (s)',
    eye=EYE_MH_W,
)

In [ ]:
plot3D_levelset(
    impl.project_onto(out, 'x', 'y', 'yaw', complete=True),
    min_bounds=[MIN_BOUNDS[0], MIN_BOUNDS[1], MIN_BOUNDS[2]],
    max_bounds=[MAX_BOUNDS[0], MAX_BOUNDS[1], MAX_BOUNDS[2]],
    axes=(0, 1, 2),
    xtitle='x (m)',
    ytitle='y (m)',
    ztitle='yaw (rad)',
    eye=EYE_MH_W,
)

## **Specification: $\square \psi$**

This specification requires the system to remain within a safe region $\psi$ for the entire time horizon. It encodes an invariance property, meaning the condition $\psi$ must hold at all times. The HJ backend realizes this using an avoid set computed via universal control, solving $\Box \psi$ as a special case of $\neg \Diamond \neg \psi$, while the HZ backend uses a fixed-point formulation, i.e. $\psi \land \bigcirc(\psi \land \bigcirc(\dots))$. Both methods result a valid set of initial states from which the system can always satisfy $\psi$ throughout the trajectory.

### **Imports**

In [ ]:
# Implementation independent

from time import time
from contextlib import contextmanager
from tqdm import tqdm

from pyspect import *

# HJ specific

from pyspect.impls.hj_reachability import TVHJImpl
from pyspect.plotting.levelsets import *

import hj_reachability as hj
from hj_reachability.systems import DoubleIntegrator as HJDoubleIntegrator

# HZ specific

from pyspect.plotting.zonotopes import _hz2hj

from hz_reachability.hz_impl import TVHZImpl
from hz_reachability.systems.cars import *
from hz_reachability.systems.integrators import DoubleIntegrator as HZDoubleIntegrator
from hz_reachability.spaces import EmptySpace

In [ ]:
@contextmanager
def timectx(msgfunc):
    """Context manager to time a block of code."""
    start = time()
    yield
    end = time()
    print(msgfunc(end-start))

def print_hzinfo(hz):
    if isinstance(hz, list):
        nz, ng, nb, nc = \
            np.array([[_out.dim, _out.ng, _out.nb, _out.nc]
                    for _out in hz]).max(axis=0)
    else:
        nz,ng,nb,nc = hz.dim, hz.ng, hz.nb, hz.nc
    print(f"nz: {nz}, ng: {ng}, nb: {nb}, nc: {nc}")

### **Hyperparameters**

In [ ]:
AXIS_NAMES = ['x',  'v'] # [m, m/s]
MAX_BOUNDS = [+100, +20] # 500m, 30 mps ~= 110 kmph
MIN_BOUNDS = [-100, -20] #   0m,  0 mps
GRID_SHAPE = (  91,  91)

MAX_ACCEL = 1.0     # [mps2]
TIME_STEP = 0.5     # [s]
TIME_HORIZON = 40   # [s]

### **Program**

In [ ]:
## SPECIFICATION

T = BoundedSet(x=(-50,  +50))

phi = ALWAYS(T)

#### Two interpretations of ALWAYS

In [ ]:
# Define ALWAYS through RCI set (relating to ¬◇¬ψ)

@primitive(ALWAYS('_1'))
def Always_rci(_1: 'TLTLike') -> Tuple[SetBuilder, APPROXDIR]:
    b1, a1 = _1._builder, _1._approx

    ao = APPROXDIR.UNDER
    return (
        AppliedSet('rci', b1),
        ao + a1 if ao * a1 == APPROXDIR.EXACT else 
        a1      if ao == a1 else
        APPROXDIR.INVALID,
    )

# Define Always through fixed-point iteration

@primitive(ALWAYS('_1'))
def Always_fp(_1: 'TLTLike') -> Tuple[SetBuilder, APPROXDIR]:

    N = int(TIME_HORIZON / TIME_STEP)

    phi = 'psi'
    for _ in range(N-1):
        phi = AND('psi', NEXT(phi))

    tree = TLT(phi).where(psi=_1)
    return (tree._builder, tree._approx)

#### HJ Implementation

In [ ]:
## CONSTRUCT TLT

TLT.select(LTLc | Always_rci)

tree = TLT(phi)

print(f"Approximation direction: {tree._approx = }")

## INITIALIZE IMPLEMENTATION

dynamics = dict(cls=HJDoubleIntegrator,
                min_accel=-MAX_ACCEL,
                max_accel=+MAX_ACCEL)

impl = TVHJImpl(dynamics, 
                AXIS_NAMES,
                MIN_BOUNDS,
                MAX_BOUNDS,
                GRID_SHAPE,
                TIME_HORIZON,
                time_step=TIME_STEP)

with timectx(lambda t: f"Realization with HJ took {t:.2f} seconds"):
    out = tree.realize(impl)

print(f"{type(out) = }")

## PLOT

plot3D_levelset(
    out,
    min_bounds=[           0, *MIN_BOUNDS],
    max_bounds=[TIME_HORIZON, *MAX_BOUNDS],
    xtitle='Position (m)',
    ytitle='Velocity (m/s)',
    colorscale='greens',
    eye=EYE_MH_W,
)

#### HZ Implementation

In [ ]:
## CONSTRUCT TLT

TLT.select(LTLd | Always_fp)

tree = TLT(phi)

print(f"Approximation direction: {tree._approx = }")

## INITIALIZE IMPLEMENTATION

space = EmptySpace(MIN_BOUNDS, MAX_BOUNDS)

dynamics = HZDoubleIntegrator(max_accel=MAX_ACCEL, dt=TIME_STEP)

impl = TVHZImpl(dynamics, space, AXIS_NAMES, time_horizon=TIME_HORIZON, time_step=TIME_STEP)
# impl.enable_reduce = True

with timectx(lambda t: f"Realization with HZ took {t:.2f} seconds"):
    out = tree.realize(impl)

print(f"{type(out) = }")
print_hzinfo(out)

## Plot

if not isinstance(out, list):
    vf = np.array([_hz2hj(out, MIN_BOUNDS, MAX_BOUNDS, GRID_SHAPE)] * impl.N)
else:
    vf = np.array([_hz2hj(_out, MIN_BOUNDS, MAX_BOUNDS, GRID_SHAPE) for _out in tqdm(out)])

plot3D_levelset(
    vf,
    min_bounds=[           0, *MIN_BOUNDS],
    max_bounds=[TIME_HORIZON, *MAX_BOUNDS],
    xtitle='Position (m)',
    ytitle='Velocity (m/s)',
    colorscale='blues',
    eye=EYE_MH_W,
)

## **Specification: $\square \lozenge \psi$**

This specification ensures that the system can reach $\psi$ infinitely often, i.e., $\psi$ must remain recurrently reachable throughout the time horizon. Formally, it is a liveness property requiring that at every point in time, it is possible to reach a state satisfying $\psi$ in the future. The HJ backend supports this through nested reachability using negation and disjunction (e.g., $\Box \Diamond \psi = \neg \Diamond \neg \Diamond \psi$). However, HZ implementations typically cannot soundly support this due to approximation mismatches may violate the logical semantics when doing the fixed-point iteration. In response, pyspect flags and rejects this combination.

### **Imports**

In [ ]:
# Implementation independent

from time import time
from contextlib import contextmanager
from tqdm import tqdm

from pyspect import *

# HJ specific

from pyspect.impls.hj_reachability import TVHJImpl
from pyspect.plotting.levelsets import *

import hj_reachability as hj
from hj_reachability.systems import DoubleIntegrator as HJDoubleIntegrator

# HZ specific

from pyspect.plotting.zonotopes import _hz2hj

from hz_reachability.hz_impl import TVHZImpl
from hz_reachability.systems.cars import *
from hz_reachability.systems.integrators import DoubleIntegrator as HZDoubleIntegrator
from hz_reachability.spaces import EmptySpace

In [ ]:
@contextmanager
def timectx(msgfunc):
    """Context manager to time a block of code."""
    start = time()
    yield
    end = time()
    print(msgfunc(end-start))

def print_hzinfo(hz):
    if isinstance(hz, list):
        nz, ng, nb, nc = \
            np.array([[_out.dim, _out.ng, _out.nb, _out.nc]
                    for _out in hz]).max(axis=0)
    else:
        nz,ng,nb,nc = hz.dim, hz.ng, hz.nb, hz.nc
    print(f"nz: {nz}, ng: {ng}, nb: {nb}, nc: {nc}")

### **Hyperparameters**

In [ ]:
AXIS_NAMES = ['x',  'v'] # [m, m/s]
MAX_BOUNDS = [+100, +20] # 500m, 30 mps ~= 110 kmph
MIN_BOUNDS = [-100, -20] #   0m,  0 mps
GRID_SHAPE = (  91,  91)

MAX_ACCEL = 1.0     # [mps2]
TIME_STEP = 0.5     # [s]
TIME_HORIZON = 40   # [s]

### **Program**

In [ ]:
## SPECIFICATION

T = BoundedSet(x=(-50,  +50))

phi = ALWAYS(EVENTUALLY(T))

#### Two interpretations of ALWAYS

In [ ]:
# Define ALWAYS through RCI set (relating to ¬◇¬ψ)

@primitive(ALWAYS('_1'))
def Always_rci(_1: 'TLTLike') -> Tuple[SetBuilder, APPROXDIR]:
    b1, a1 = _1._builder, _1._approx

    ao = APPROXDIR.UNDER
    return (
        AppliedSet('rci', b1),
        ao + a1 if ao * a1 == APPROXDIR.EXACT else 
        a1      if ao == a1 else
        APPROXDIR.INVALID,
    )

# Define Always through fixed-point iteration

@primitive(ALWAYS('_1'))
def Always_fp(_1: 'TLTLike') -> Tuple[SetBuilder, APPROXDIR]:

    N = int(TIME_HORIZON / TIME_STEP)

    phi = 'psi'
    for _ in range(N-1):
        phi = AND('psi', NEXT(phi))

    tree = TLT(phi).where(psi=_1)
    return (tree._builder, tree._approx)

#### HJ Implementation

In [ ]:
## CONSTRUCT TLT

TLT.select(LTLc | Always_rci)
tree = TLT(phi)

## INITIALIZE IMPLEMENTATION

dynamics = dict(cls=HJDoubleIntegrator,
                min_accel=-MAX_ACCEL,
                max_accel=+MAX_ACCEL)

impl = TVHJImpl(dynamics, 
                AXIS_NAMES,
                MIN_BOUNDS,
                MAX_BOUNDS,
                GRID_SHAPE,
                TIME_HORIZON,
                time_step=TIME_STEP)

with timectx(lambda t: f"Realization with HJ took {t:.2f} seconds"):
    out = tree.realize(impl)

## Plot

plot3D_levelset(
    out,
    min_bounds=[           0, *MIN_BOUNDS],
    max_bounds=[TIME_HORIZON, *MAX_BOUNDS],
    xtitle='Position (m)',
    ytitle='Velocity (m/s)',
    colorscale='greens',
    eye=EYE_MH_W,
)

#### HZ Implementation

In [ ]:
## CONSTRUCT TLT

TLT.select(LTLd | Always_fp)
tree = TLT(phi)

## INITIALIZE IMPLEMENTATION

space = EmptySpace(MIN_BOUNDS, MAX_BOUNDS)

dynamics = HZDoubleIntegrator(max_accel=MAX_ACCEL, dt=TIME_STEP)

impl = TVHZImpl(dynamics, space, AXIS_NAMES, time_horizon=TIME_HORIZON, time_step=TIME_STEP)
# impl.enable_reduce = True

with timectx(lambda t: f"Realization with HZ took {t:.2f} seconds"):
    out = tree.realize(impl)

print_hzinfo(out)

## Plot

if not isinstance(out, list):
    vf = np.array([_hz2hj(out, MIN_BOUNDS, MAX_BOUNDS, GRID_SHAPE)] * impl.N)
else:
    vf = np.array([_hz2hj(_out, MIN_BOUNDS, MAX_BOUNDS, GRID_SHAPE) for _out in tqdm(out)])

plot3D_levelset(
    vf,
    min_bounds=[           0, *MIN_BOUNDS],
    max_bounds=[TIME_HORIZON, *MAX_BOUNDS],
    xtitle='Position (m)',
    ytitle='Velocity (m/s)',
    colorscale='blues',
    eye=EYE_MH_W,
)